In [1]:
#Required Imports
import numpy as np 
import pandas as pd 
import requests 
print('Import Complete')

Import Complete


In [2]:
#Required Imports
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
print('Import Complete')

Import Complete


In [3]:
Wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Bea_soup = BeautifulSoup(Wiki.text, 'lxml')

#Getting the data from HTML and converting into a table using Wiki Soup

data = []
columns = []
table = Bea_soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #Making it known that the first row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#Pandas dataframe conversion
toto_df = pd.DataFrame(data = data,columns = columns)

In [4]:
toto_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
toto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code     180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [6]:
#Data Cleanup
#Removing Boroughs that are listed as 'not assigned'

toto_df = toto_df[toto_df['Borough'] != 'Not assigned']

In [7]:

toto_df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
#Removing Duplicates from the dataset
toto_df['Borough']= toto_df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [10]:
#Dropped Duplicates
toto_df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,NorthYork,Parkwoods
3,M4A,NorthYork,Victoria Village
4,M5A,DowntownToronto,"Regent Park, Harbourfront"
5,M6A,NorthYork,"Lawrence Manor, Lawrence Heights"
6,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,NorthYork,Don Mills
12,M4B,EastYork,"Parkview Hill, Woodbine Gardens"
13,M5B,DowntownToronto,"Garden District, Ryerson"


In [11]:
#Updated DF info
toto_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postal Code     103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [12]:
toto_df.shape

(103, 3)

## Part 2 Obtaining Latitude and Longitude

In [13]:
#Identify Location in Toronto Test
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 124974741, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [14]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates are downloaded')
torogla = pd.read_csv('toronto_coordinates.csv')

Coordinates are downloaded


In [15]:
print(torogla.shape)
print(torogla.info)
torogla.head(5)

(103, 3)
<bound method DataFrame.info of     Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
5           M1J  43.744734 -79.239476
6           M1K  43.727929 -79.262029
7           M1L  43.711112 -79.284577
8           M1M  43.716316 -79.239476
9           M1N  43.692657 -79.264848
10          M1P  43.757410 -79.273304
11          M1R  43.750072 -79.295849
12          M1S  43.794200 -79.262029
13          M1T  43.781638 -79.304302
14          M1V  43.815252 -79.284577
15          M1W  43.799525 -79.318389
16          M1X  43.836125 -79.205636
17          M2H  43.803762 -79.363452
18          M2J  43.778517 -79.346556
19          M2K  43.786947 -79.385975
20          M2L  43.757490 -79.374714
21          M2M  43.789053 -79.408493
22          M2N  43.770120 -79.408493
23          M2P  43.752758 -79.400049
24       

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# Merger of Dataframes

ronto_temp = toto_df.set_index('Postal Code')
kors_temp = torogla.set_index('Postal Code')
ronto_coors_df = pd.concat([ronto_temp, kors_temp], axis=1, join='inner')

ronto_coors_df.index.name = 'Postal Code'
ronto_coors_df.reset_index(inplace=True)

print(ronto_coors_df.shape)
ronto_coors_df.head(15)

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,Victoria Village,43.725882,-79.315572
2,M5A,DowntownToronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,NorthYork,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,NorthYork,Don Mills,43.745906,-79.352188
8,M4B,EastYork,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,DowntownToronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3 Examining Locations in Toronto

In [17]:
#Locating Geographic Location of Toronto

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Coordinates of Toronto are 43.6534817, -79.3839347.


### Mapping With Folium

In [18]:
# Folium Install
!conda install -c conda-forge folium=0.5.0 

Solving environment: done

# All requested packages already installed.



In [20]:
#Necessary Imports

import pandas as pd
import folium
print('Import Complete')

Import Complete


In [23]:
#Viewing the Map of Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, long, post, borough, neigh in zip(ronto_coors_df['Latitude'], ronto_coors_df['Longitude'], ronto_coors_df['Postal Code'], ronto_coors_df['Borough'], ronto_coors_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=popup,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(toronto_map)
    
toronto_map

In [24]:
# Viewing of Long. AND Lat. From Dataframe
ronto_coors_df.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,NorthYork,Parkwoods,43.753259,-79.329656
1,M4A,NorthYork,Victoria Village,43.725882,-79.315572
2,M5A,DowntownToronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,NorthYork,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,DowntownToronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,NorthYork,Don Mills,43.745906,-79.352188
8,M4B,EastYork,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,DowntownToronto,"Garden District, Ryerson",43.657162,-79.378937


## Foursquare Exploration with API

In [25]:
CLIENT_ID = 'GJE304UFKEIN5QUWCG4FDYKGAVCP335MZXYVPDKXGWWU04YU' 
CLIENT_SECRET = 'YNKPWJFUCZGZTOA0OMC3QQYN1SITXOQ51TFHIUAHUYPM2QHV'
VERSION = '20190330' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GJE304UFKEIN5QUWCG4FDYKGAVCP335MZXYVPDKXGWWU04YU
CLIENT_SECRET:YNKPWJFUCZGZTOA0OMC3QQYN1SITXOQ51TFHIUAHUYPM2QHV


In [34]:
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [39]:
# create map of Toronto using latitude and longitude values
ronto_neighbourhoods_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(ronto_coors_df['Latitude'], ronto_coors_df['Longitude'], ronto_coors_df['Borough'], ronto_coors_df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
ronto_neighbourhoods_map

ronto_neighbourhoods_map.save("map_toronto_neighbourhoods.html")



## Etobicoke Venue Analysis

In [41]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Etobicoke, Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinates of Etobicoke, Toronto are 43.671459150000004, -79.55249206611668.


In [44]:
#Etobicoke Data Venue list
etobicoke_data = ronto_coors_df[ronto_coors_df['Borough'] == 'Etobicoke'].reset_index(drop=True)
etobicoke_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
3,M9P,Etobicoke,Westmount,43.696319,-79.532242
4,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
5,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
6,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
7,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
8,M9W,Etobicoke,Northwest,43.706748,-79.594054
9,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


In [46]:
etobicoke_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
Postal Code     12 non-null object
Borough         12 non-null object
Neighborhood    12 non-null object
Latitude        12 non-null float64
Longitude       12 non-null float64
dtypes: float64(2), object(3)
memory usage: 560.0+ bytes


In [50]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Postal Code'], etobicoke_data['Borough'], etobicoke_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [54]:
df_venues = pd.DataFrame(venues)
df_venues.columns = ['Postal Code', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(df_venues.shape)
df_venues.head(5)

(68, 9)


,Postal Code,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,LCBO,43.642099,-79.576592,Liquor Store
1,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Starbucks,43.641312,-79.576924,Coffee Shop
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,The Beer Store,43.641313,-79.576925,Beer Store
3,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Shoppers Drug Mart,43.641312,-79.576924,Cosmetics Shop
4,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,Pizza Hut,43.641845,-79.576556,Pizza Place


In [55]:
df_venues.groupby(['Postal Code', 'Borough', 'Neighborhood'])['VenueName'].count()

Postal Code  Borough    Neighborhood                                                                                                                          
M8V          Etobicoke  New Toronto, Mimico South, Humber Bay Shores                                                                                              11
M8W          Etobicoke  Alderwood, Long Branch                                                                                                                     9
M8X          Etobicoke  The Kingsway, Montgomery Road, Old Mill North                                                                                              3
M8Y          Etobicoke  Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East     1
M8Z          Etobicoke  Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West                                            13
M9C          Eto

In [56]:
len(df_venues['VenueCategory'].unique())

39

##  Venue Frequency Analysis

In [65]:
#Creating a graph that identifies Venue Frequency
ronto_hot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")


ronto_hot['Postal Code'] = venues_df['Postal Code'] 
ronto_hot['Borough'] = venues_df['Borough'] 
ronto_hot['Neighborhoods'] = venues_df['Neighborhood'] 


fixed_columns = list(ronto_hot.columns[-3:]) + list(ronto_hot.columns[:-3])
ronto_hot = ronto_hot[fixed_columns]

print(ronto_hot.shape)
ronto_hot.head(5)

(68, 42)


,Postal Code,Borough,Neighborhoods,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Wings Joint
0,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
rontos_freq = ronto_hot.groupby(['Postal Code', 'Borough', 'Neighborhoods']).mean().reset_index()
print(rontos_freq.shape)
rontos_freq.head()

(10, 42)


,Postal Code,Borough,Neighborhoods,American Restaurant,Bakery,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,...,Pub,Rental Car Location,Restaurant,River,Sandwich Place,Skating Rink,Smoke Shop,Social Club,Supplement Shop,Wings Joint
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",0.090909,0.090909,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M8W,Etobicoke,"Alderwood, Long Branch",0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.111111,0.0,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.000000
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.076923,0.0,0.0,0.0,0.076923,0.076923,...,0.000000,0.0,0.000000,0.000000,0.076923,0.000000,0.000000,0.076923,0.076923,0.076923
